#### MAML pretrain  
num_epochs=100<br>   
inner_lr: 0.01 / 0.005 / 0.001 / 0.0005 <br>
meta_lr :  0.0005 / 0.0004 / 0.0003 / 0.0003 / 0.0001 <br>
meta_loss_sum / K : True, False <br>
valid loader use : True, False <br>
Activation function : [relu, sigmoid] <br>


#### Pretrain
epochs=200,<br> lr=0.0001<br> <br>
Sigmoid

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict
# from src.tasks import Sine_Task, Sine_Task_Distribution
import matplotlib.pyplot as plt
from models.utils import *
import os
import datetime
import copy
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from lifelines.utils import concordance_index
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.nn.parameter import Parameter
from torchsummary import summary as summary
# from models.tasks import Workload_Task
# from pytorchtools import EarlyStopping

# interpreter : py3.8

## Data
data load and preprocessing

In [ ]:
# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)

sample_path = 'data/rocksdb_benchmark_'
batch_size = 32
hidden_dim = 16
output_dim = 1
group_dim = 32
wk_vec_dim = 4
metrics = ['TIME', 'RATE', 'WAF', 'SA']
wk_vec = [[9,1,0,1], [9,1,0,4], [9,1,0,16], [9,1,0,64],
            [1,1,0,1], [1,1,0,4], [1,1,0,16], [1,1,0,64],
            [1,9,0,1], [1,9,0,4], [1,9,0,16], [1,9,0,64],
            [0,0,1,1], [0,0,1,4], [0,0,1,16], [0,0,1,64]]

wk_using_training_step = [0,1,3,
                          4,5,7,
                          8,9,11,
                          12,13,15]
K_tr = 32   # batch size of training step
# K_te = 10   # batch size of testing step
# wk_using_testing_step = [2,6,10,14]

def raw_data_make(metrics, wk_vec, sample_path='data/rocksdb_benchmark_'):
    entire_X_tr = pd.DataFrame()
    entire_X_te = pd.DataFrame()
    entire_y_tr = pd.DataFrame()
    entire_y_te = pd.DataFrame()

    for wk in range(16):
        raw_data = pd.read_csv(sample_path+f'{wk}.csv')
        y = raw_data[['RATE']]
        x = raw_data.drop(columns=metrics)
        pd_wk_info = pd.DataFrame(data=np.repeat([wk_vec[wk]], x.shape[0], axis=0), columns=["READ", "WRITE", "UPDATE", "VALUE_SIZE"])
        X = pd.concat((x, pd_wk_info), axis=1)

        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1004)
        entire_X_tr = pd.concat((entire_X_tr, X_tr))
        entire_X_te = pd.concat((entire_X_te, X_te))
        entire_y_tr = pd.concat((entire_y_tr, y_tr))
        entire_y_te = pd.concat((entire_y_te, y_te))

    return entire_X_tr, entire_X_te, entire_y_tr, entire_y_te

entire_X_tr, entire_X_te, entire_y_tr, entire_y_te = raw_data_make(metrics, wk_vec, sample_path)

scaler_X = MinMaxScaler().fit(entire_X_tr)

#scaler_X = entire_X_tr
scaler_y = StandardScaler().fit(entire_y_tr)

input_dim = entire_X_tr.shape[-1]

# K = 32  # batch size



# WK_NUM = 16
# EX_NUM=4

Make training step dataloader

In [ ]:
# for MAML model dataloader in training step
Train_task_MAML_DL_tr, Train_task_MAML_DL_te, Train_task_MAML_X_te, Train_task_MAML_y_te = MAML_dataset(entire_X_tr, entire_y_tr, entire_X_te, entire_y_te, scaler_X, scaler_y, wk_using_training_step, batch_size=K_tr)

###################################################################
Train_task_MAML_DL_val, _, _, _ = MAML_dataset(entire_X_tr, entire_y_tr, entire_X_te, entire_y_te, scaler_X, scaler_y, wk_using_training_step, batch_size=K_tr)
###################################################################

# for pretrain model dataloader in training step
Train_task_pretrain_DL_tr, Train_task_pretrain_DL_te, Train_task_pretrain_X_te, Train_task_pretrain_y_te = pretrain_dataset(entire_X_tr, entire_y_tr, entire_X_te, entire_y_te, scaler_X, scaler_y, wk_using_training_step, batch_size=K_tr)

## Model

To implement MAML in Pytorch, we will need a model that can be easily parametrised with a set of weights that's distinct from the model's own parameters. Having these weights distinct from the model.parameters() allows us to easily make differentiable gradient updates within the inner loop.

In [ ]:
'''ReshapeNet - activation : ReLU'''
class ReshapeNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=16, group_dim=32, wk_vec_dim=4, output_dim=1):
        super(ReshapeNet, self).__init__()
        self.input_dim = input_dim - wk_vec_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.group_dim = group_dim
        self.wk_vec_dim = wk_vec_dim

        self.embedding = nn.Linear(self.wk_vec_dim, self.hidden_dim)
        #self.knob_fc = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim*self.group_dim), nn.Sigmoid()) # (22, 1) -> (group*hidden, 1)
        self.knob_fc = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim*self.group_dim), nn.ReLU()) # (22, 1) -> (group*hidden, 1)
        self.attention = nn.MultiheadAttention(self.hidden_dim, 1)
        #self.active = nn.Sigmoid()
        self.activate = nn.ReLU()
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        wk = x[:, -self.wk_vec_dim:] # only workload information
        x = x[:, :-self.wk_vec_dim] # only knobs
        
        self.embed_wk = self.embedding(wk) # (batch, 4) -> (batch, dim)
        self.embed_wk = self.embed_wk.unsqueeze(1) # (batch, 1, dim)
        self.x = self.knob_fc(x) # (batch, 22) -> (batch, group*hidden)
        self.res_x = torch.reshape(self.x, (-1, self.group_dim, self.hidden_dim)) # (batch, group, hidden)
        
        # attn_ouptut = (1, batch, hidden), attn_weights = (batch, 1, group)
        self.attn_output, self.attn_weights = self.attention(self.embed_wk.permute((1,0,2)), self.res_x.permute((1,0,2)), self.res_x.permute((1,0,2)))
        self.attn_output = self.activate(self.attn_output.squeeze())
        outs = self.attn_output
        self.outputs = self.fc(outs)  
        return self.outputs
    
    def parameterised(self, x, weights):
        # like forward, but uses ``weights`` instead of ``model.parameters()``
        # it'd be nice if this could be generated automatically for any nn.Module...
        # https://easy-going-programming.tistory.com/11
        wk = x[:, -self.wk_vec_dim:] # only workload information
        x = x[:, :-self.wk_vec_dim] # only knobs
        embed_wk = F.linear(wk, weights[0], weights[1])
        embed_wk = embed_wk.unsqueeze(1)
        x = F.linear(x, weights[2], weights[3] )    # (22, 1) -> (group*hidden, 1)
        #x = F.sigmoid(x)
        x = F.relu(x)
        self.p_res_x = torch.reshape(x, (-1, self.group_dim, self.hidden_dim)) # (batch, group, hidden)
        # attn_output, self.attn_weights = self.attention(embed_wk.permute((1,0,2)), res_x.permute((1,0,2)), res_x.permute((1,0,2)))
        attn_output, _ = F.multi_head_attention_forward(embed_wk.permute((1,0,2)), self.p_res_x.permute((1,0,2)), self.p_res_x.permute((1,0,2)), self.hidden_dim, 1, 
                                                        weights[4], weights[5], 
                                                        None, None, False, 0,
                                                        weights[6], weights[7]) # self.attention(embed_wk.permute((1,0,2)), res_x.permute((1,0,2)), res_x.permute((1,0,2)))
        #attn_output = F.sigmoid(attn_output.squeeze())
        attn_output = F.relu(attn_output.squeeze())
        outputs = F.linear(attn_output, weights[8], weights[9])

        return outputs                           

In [ ]:
# '''ReshapeNet - activation : Sigmoid'''
# class ReshapeNet(nn.Module):
#     def __init__(self, input_dim, hidden_dim=16, group_dim=32, wk_vec_dim=4, output_dim=1):
#         super(ReshapeNet, self).__init__()
#         self.input_dim = input_dim - wk_vec_dim
#         self.hidden_dim = hidden_dim
#         self.output_dim = output_dim
#         self.group_dim = group_dim
#         self.wk_vec_dim = wk_vec_dim

#         self.embedding = nn.Linear(self.wk_vec_dim, self.hidden_dim)
#         #self.knob_fc = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim*self.group_dim), nn.Sigmoid()) # (22, 1) -> (group*hidden, 1)
#         self.knob_fc = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim*self.group_dim), nn.Sigmoid()) # (22, 1) -> (group*hidden, 1)
#         self.attention = nn.MultiheadAttention(self.hidden_dim, 1)
#         #self.active = nn.Sigmoid()
#         self.activate = nn.Sigmoid()
#         self.fc = nn.Linear(self.hidden_dim, self.output_dim)

#     def forward(self, x):
#         wk = x[:, -self.wk_vec_dim:] # only workload information
#         x = x[:, :-self.wk_vec_dim] # only knobs
        
#         self.embed_wk = self.embedding(wk) # (batch, 4) -> (batch, dim)
#         self.embed_wk = self.embed_wk.unsqueeze(1) # (batch, 1, dim)
#         self.x = self.knob_fc(x) # (batch, 22) -> (batch, group*hidden)
#         self.res_x = torch.reshape(self.x, (-1, self.group_dim, self.hidden_dim)) # (batch, group, hidden)
        
#         # attn_ouptut = (1, batch, hidden), attn_weights = (batch, 1, group)
#         self.attn_output, self.attn_weights = self.attention(self.embed_wk.permute((1,0,2)), self.res_x.permute((1,0,2)), self.res_x.permute((1,0,2)))
#         self.attn_output = self.activate(self.attn_output.squeeze())
#         outs = self.attn_output
#         self.outputs = self.fc(outs)  
#         return self.outputs
    
#     def parameterised(self, x, weights):
#         # like forward, but uses ``weights`` instead of ``model.parameters()``
#         # it'd be nice if this could be generated automatically for any nn.Module...
#         # https://easy-going-programming.tistory.com/11
#         wk = x[:, -self.wk_vec_dim:] # only workload information
#         x = x[:, :-self.wk_vec_dim] # only knobs
#         embed_wk = F.linear(wk, weights[0], weights[1])
#         embed_wk = embed_wk.unsqueeze(1)
#         x = F.linear(x, weights[2], weights[3] )    # (22, 1) -> (group*hidden, 1)
#         #x = F.sigmoid(x)
#         x = self.activate(x)
#         self.p_res_x = torch.reshape(x, (-1, self.group_dim, self.hidden_dim)) # (batch, group, hidden)
#         # attn_output, self.attn_weights = self.attention(embed_wk.permute((1,0,2)), res_x.permute((1,0,2)), res_x.permute((1,0,2)))
#         attn_output, _ = F.multi_head_attention_forward(embed_wk.permute((1,0,2)), self.p_res_x.permute((1,0,2)), self.p_res_x.permute((1,0,2)), self.hidden_dim, 1, 
#                                                         weights[4], weights[5], 
#                                                         None, None, False, 0,
#                                                         weights[6], weights[7]) # self.attention(embed_wk.permute((1,0,2)), res_x.permute((1,0,2)), res_x.permute((1,0,2)))
#         #attn_output = F.sigmoid(attn_output.squeeze())
#         attn_output = self.activate(attn_output.squeeze())
#         outputs = F.linear(attn_output, weights[8], weights[9])

#         return outputs                           

## Training step of MAML,  normal pretrain


In [ ]:
class MAML_one_batch():
    def __init__(self, model, train_dataloaders, val_dataloaders, test_dataloaders, num_epochs, inner_lr, meta_lr, inner_steps=1, meta_mean=False, dot=True, lamb=0.6):
    # def __init__(self, model, train_dataloaders, test_dataloaders, meta_tasks, inner_lr, meta_lr, K=K, inner_steps=1):   # K : number of sample data of task
        
        #############################################
        self.patience = 10  # for early stopping 
        #############################################
        self.meta_mean = meta_mean

        # important objects
        self.model = model        
        self.train_dataloaders = train_dataloaders
        self.val_dataloaders = val_dataloaders
        self.test_dataloaders = test_dataloaders

        # hyperparameters
        self.num_epochs = num_epochs
        self.inner_lr = inner_lr
        self.meta_lr = meta_lr
        self.dot = dot
        self.lamb = lamb

        # self.meta_tasks = meta_tasks    # list of using workload number for MAML
        self.num_meta_tasks = len(self.train_dataloaders)    # len(train_dataloaders) = len(test_dataloaders) 
        self.criterion = nn.MSELoss()
        self.weights = list(self.model.parameters()) # the maml weights we will be meta-optimising
        # ######################################################
        # # self.weights = list(torch.nn.init.xavier_uniform_(self.model.parameters())) # the maml weights we will be meta-optimising
        # self.weights = list(self.model.parameters()) # the maml weights we will be meta-optimising
        # for i in range(len(self.weights)):
        #     self.weights[i]=torch.nn.init.kaiming_normal_(self.weights[i])
        # ######################################################
        self.meta_optimizer = torch.optim.Adam(self.weights, self.meta_lr)
        
        # self.K = K
        self.inner_steps = inner_steps # with the current design of MAML, >1 is unlikely to work well         
        
        # metrics
        self.meta_losses_tr = []
        self.meta_losses_te = []
        self.r2_score = []
        self.best_loss = np.inf
        # self.best_loss = 

    
    def inner_loop(self, iter):     # i: task , iteration : iteration
        # reset inner model to current maml weights
        temp_weights = [w.clone() for w in self.weights]         
        # perform training on data sampled from task

        X, y = self.sample_tr[0], self.sample_tr[1]
        ##########################################################
        X_val, y_val = self.sample_val[0], self.sample_val[1]
        ###########################################################
        inner_loss = self.criterion(self.model.parameterised(X, temp_weights), y)
        grad = torch.autograd.grad(inner_loss, temp_weights)
        temp_weights = [w - self.inner_lr * g for w, g in zip(temp_weights, grad)]

        temp_pred = self.model.parameterised(X_val, temp_weights)
        # calculate loss for update maml weight (with update inner loop weight)
        if self.dot:
            d = torch.bmm(self.model.p_res_x, self.model.p_res_x.transpose(1, 2))
            dot_loss = F.mse_loss(d, torch.eye(d.size(1)).repeat(X.shape[0], 1, 1).cuda())
            meta_loss = (1-self.lamb)*self.criterion(temp_pred, y) + self.lamb*dot_loss
            # meta_loss = (1-self.lamb)*F.mse_loss(self.model.parameterised(X, temp_weights), y) + self.lamb*dot_loss
        else:
            meta_loss = self.self.criterion(temp_pred, y_val)
        
        return inner_loss, meta_loss

    def main_loop(self):
        # epoch_loss = 0 ####
        trigger_times = 0
        breaker = False ####
        for e in range(1, self.num_epochs+1):    # epoch
            sampler_tr = Sampler(dataloaders=self.train_dataloaders)
            #################################################################
            sampler_val = Sampler(dataloaders=self.val_dataloaders)
            #################################################################    
            for iter in range(1, len(self.train_dataloaders[0])+1):    # iteration       
                total_meta_loss_tr = 0
   
                self.meta_optimizer.zero_grad()
                sample_tr = sampler_tr.get_sample()
                sample_val = sampler_val.get_sample()
                meta_loss_sum = 0
                wk_loss =[]                     
                for num_wk in range(self.num_meta_tasks):
                    self.sample_tr = sample_tr[num_wk]
                    self.sample_val = sample_val[num_wk]  ###                  
                    _, meta_loss = self.inner_loop(iter)
                    wk_loss.append(meta_loss)
                    meta_loss_sum += meta_loss   # i: task                               
                # print(f'meta_loss_sum : {meta_loss_sum}')   ####
                if self.meta_mean ==True :
                    meta_loss_sum /= self.num_meta_tasks    ####
                # print(f'meta_loss_mean : {meta_loss_sum}')  ####
                total_meta_loss_tr += meta_loss_sum.item()
                meta_loss_tr = total_meta_loss_tr/len(self.train_dataloaders)
                
                # compute meta gradient of loss with respect to maml weights
                meta_loss_sum.backward()
                self.meta_optimizer.step()
            loss_te, te_outputs, r2_res = self.validate(model)  ###

            # self.meta_losses_tr +=[meta_loss_sum]
            self.meta_losses_tr +=[meta_loss_tr]
            self.meta_losses_te += [loss_te]       
            self.r2_score += [r2_res]      

            time_ = datetime.datetime.today().strftime('%y%m%d/%H:%M:%S')
            print(f"{time_}[{e:02d}/{self.num_epochs}] meta_loss: {meta_loss_sum:.4f} loss_te: {loss_te:.4f}, r2_res = {r2_res:.4f}")

            # print(f"{time_}[{e:02d}/{self.num_epochs}] meta_loss: {meta_loss_sum:.4f} loss_te: {loss_te:.4f}, r2_res = {r2_res:.4f} / trrigger times : {trigger_times}")

            # if loss_te < self.best_loss:
            #     self.best_loss = loss_te
            #     self.best_ouputs = te_outputs
            #     self.best_r2_score = r2_res
            #     self.best_model = self.model
            #     self.best_epoch_num = e

            ###############################################################
            # early stopping
            if loss_te > self.best_loss:
                trigger_times += 1
                # print('Trigger Times:' , trigger_times)
                if trigger_times >= self.patience:
                    print('Early stopping! \n training step finish')
                    breaker = True
            else:
                self.best_loss = loss_te
                self.best_ouputs = te_outputs
                self.best_r2_score = r2_res
                self.best_model = self.model
                self.best_epoch_num = e
                print('Trigger Times: 0')
                trigger_times = 0  

            print(f"{time_}[{e:02d}/{self.num_epochs}] meta_loss: {meta_loss_sum:.4f} loss_te: {loss_te:.4f}, r2_res = {r2_res:.4f} / trrigger times : {trigger_times}")  

            if breaker == True:
                break
            ###############################################################


        self.name = get_filename('model_save', 'MAML_pretrain', '.pt')
        torch.save(self.best_model.state_dict(), os.path.join('model_save', self.name))  # save only state_dict of model
        print(f'Saved model state dict! name : {self.name}')
        # torch.save(self.best_model, os.path.join('model_save', name))             # save entire model


    def validate(self, model):  ###
        model.eval()

        total_loss = 0
        total_dot_loss = 0        
        outputs = torch.Tensor().cuda()
        r2_res = 0
        with torch.no_grad():
            for wk_valid_loader in self.test_dataloaders:
                for data, target in wk_valid_loader:
                    output = model(data)
                    if self.dot:
                        d = torch.bmm(model.res_x, model.res_x.transpose(1, 2))
                        dot_loss = F.mse_loss(d, torch.eye(d.size(1)).repeat(data.shape[0], 1, 1).cuda())
                        loss = (1-self.lamb)*F.mse_loss(output, target) + self.lamb*dot_loss
                    else:
                        dot_loss = 0
                        loss = F.mse_loss(output, target)
                    true = target.cpu().detach().numpy().squeeze()
                    pred = output.cpu().detach().numpy().squeeze()
                    r2_res += r2_score(true, pred)
                    total_loss += loss.item()
                    total_dot_loss += dot_loss.item()
                    outputs = torch.cat((outputs, output))
        total_loss /= len(wk_valid_loader) * len(self.test_dataloaders)
        total_dot_loss /= len(wk_valid_loader) * len(self.test_dataloaders)
        r2_res /= len(wk_valid_loader) * len(self.test_dataloaders)

        return total_loss, outputs, r2_res

In [ ]:
class nonMAML_pretrain():
    def __init__(self, model, train_dataloaders, test_dataloaders, epochs, lr, dot=True, lamb=0.6):
    # def __init__(self, model, train_dataloaders, test_dataloaders, meta_tasks, inner_lr, meta_lr, K=K, inner_steps=1):   # K : number of sample data of task
       
        #############################################
        self.patience = 10  # for early stopping 
        #############################################
        
        # important objects
        self.model = model
               
        self.train_dataloaders = train_dataloaders
        self.test_dataloaders = test_dataloaders
        # hyperparameters
        self.epochs = epochs
        self.lr = lr
        self.dot = dot
        self.lamb = lamb

        self.criterion = nn.MSELoss()
        self.weights = list(self.model.parameters()) 
        self.optimizer = torch.optim.Adam(self.weights, self.lr)      
            
        # metrics
        self.losses_tr = []
        self.losses_te = []
        self.r2_score = []
        # self.best_loss = 1000
        self.best_loss = np.inf
    
    def train(self):                
        trigger_times = 0
        breaker = False ####
        for e in range(self.epochs):    # epoch     
            for data, target in self.train_dataloaders:    
                model.zero_grad()             
                total_loss_tr = 0
                output = model(data)
                # loss type
                if self.dot:
                    d = torch.bmm(model.res_x, model.res_x.transpose(1, 2))
                    dot_loss = F.mse_loss(d, torch.eye(d.size(1)).repeat(data.shape[0], 1, 1).cuda())
                    batch_loss_tr = (1-self.lamb)*F.mse_loss(output, target) + self.lamb*dot_loss
                else:
                    dot_loss = 0
                    batch_loss_tr = F.mse_loss(output, target)                

                batch_loss_tr.backward()
                self.optimizer.step()
                total_loss_tr += batch_loss_tr.item()
            loss_tr = total_loss_tr/len(self.train_dataloaders)

            loss_te, te_outputs, r2_res = self.validate(model)  ###
            self.losses_tr += [loss_tr]
            self.losses_te += [loss_te]       
            self.r2_score += [r2_res]      
            ephch = e + 1
            time_ = datetime.datetime.today().strftime('%y%m%d/%H:%M:%S')
            # print(f"{time_}[{ephch:02d}/{self.epochs}] loss_tr: {loss_tr:.8f} loss_te: {loss_te:.8f}, r2_res = {r2_res}")

            if loss_te > self.best_loss:
                trigger_times += 1
                # print('Trigger Times:' , trigger_times)
                if trigger_times >= self.patience:
                    print('Early stopping! \n training step finish')
                    breaker = True
            else:
                self.best_loss = loss_te
                self.best_ouputs = te_outputs
                self.best_r2_score = r2_res
                self.best_model = self.model
                self.best_epoch_num = e
                print('Trigger Times: 0')
                trigger_times = 0  

            print(f"{time_}[{e:02d}/{self.num_epochs}] loss_tr: {loss_tr:.4f} loss_te: {loss_te:.4f}, r2_res = {r2_res:.4f} / trrigger times : {trigger_times}")  

            if breaker == True:
                break

            # if loss_te < self.best_loss:
            #     self.best_loss = loss_te
            #     self.best_ouputs = te_outputs
            #     self.best_r2_score = r2_res
            #     self.best_model = self.model
            #     self.best_epoch_num = e
        self.name = get_filename('model_save', 'pretrain', '.pt')
        torch.save(self.best_model.state_dict(), os.path.join('model_save', self.name))  # save only state_dict of model
        print(f'Saved model state dict! name : {self.name}')
        # torch.save(self.best_model, os.path.join('model_save', name))             # save entire model

        # return self.best_model

    def validate(self, model):  ###
        model.eval()

        total_loss = 0
        total_dot_loss = 0        
        outputs = torch.Tensor().cuda()
        r2_res = 0
        with torch.no_grad():
            # for wk_valid_loader in self.test_dataloaders:
            # for data, target in wk_valid_loader:
            for data, target in self.test_dataloaders:
                output = model(data)
                if self.dot:
                    d = torch.bmm(model.res_x, model.res_x.transpose(1, 2))
                    dot_loss = F.mse_loss(d, torch.eye(d.size(1)).repeat(data.shape[0], 1, 1).cuda())
                    
                    loss = (1-self.lamb)*F.mse_loss(output, target) + self.lamb*dot_loss
                else:
                    dot_loss = 0
                    loss = F.mse_loss(output, target)
                true = target.cpu().detach().numpy().squeeze()
                pred = output.cpu().detach().numpy().squeeze()
                r2_res += r2_score(true, pred)
                total_loss += loss.item()
                total_dot_loss += dot_loss.item()
                outputs = torch.cat((outputs, output))
        total_loss /= len(self.test_dataloaders)
        total_dot_loss /= len(self.test_dataloaders)
        r2_res /= len(self.test_dataloaders)

        return total_loss, outputs, r2_res

In [ ]:
# MAML방식 pretrain
model = ReshapeNet(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, group_dim=group_dim, wk_vec_dim=wk_vec_dim).cuda()
maml = MAML_one_batch(model, Train_task_MAML_DL_tr, Train_task_MAML_DL_te, num_epochs=200, inner_lr=0.0005, meta_lr=0.0001)
# maml = MAML_one_batch(model, Train_task_MAML_DL_tr, Train_task_MAML_DL_te, num_epochs=200, inner_lr=0.0001, meta_lr=0.0005)   # inner_lr=0.0001, meta_lr=0.0005
# maml = MAML_one_batch(model, Train_task_MAML_DL_tr, Train_task_MAML_DL_te, num_epochs=200, inner_lr=0.001, meta_lr=0.0001)    # inner_lr=0.001, meta_lr=0.0001
# maml = MAML_one_batch(model, Train_task_MAML_DL_tr, Train_task_MAML_DL_te, num_epochs=200, inner_lr=0.001, meta_lr=0.0005)    # inner_lr=0.001, meta_lr=0.0005

maml.main_loop()
name_MAML_one_batch_pt = maml.name

In [ ]:
# 일반적인 학습의 pretrain
model = ReshapeNet(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, group_dim=group_dim, wk_vec_dim=wk_vec_dim).cuda()
pretrained_model = nonMAML_pretrain(model, Train_task_pretrain_DL_tr, Train_task_pretrain_DL_te, epochs=200, lr=0.0001)
# pretrained_model = nonMAML_pretrain(model, Train_task_pretrain_DL_tr, Train_task_pretrain_DL_te, epochs=200, lr=0.0005)   # lr=0.0005

pretrained_model.train()
name_nonMAML_pretrain_pt = pretrained_model.name

In [ ]:
print(f'-save model- name : {name_MAML_one_batch_pt}')
print(f'[MAML training step]    best epoch : {maml.best_epoch_num} / best loss_te : {maml.best_loss:.4f} / best r2 : {maml.best_r2_score:.4f}')
print('-----------------------------------------------------------------------------------------')
print(f'-save model- name : {name_nonMAML_pretrain_pt}')
print(f'[NonMAML training step] best epoch : {pretrained_model.best_epoch_num} / best loss_te : {pretrained_model.best_loss:.4f} / best r2 : {pretrained_model.best_r2_score:.4f}')

Training step result plotting

In [ ]:
# def plot_training_step_result(MAML_loss_tr, MAML_loss_te, MAML_r2, pre_loss_tr, pre_loss_te, pre_r2):
#         # 그래프 배열 지정
#         f, axes = plt.subplots(2,2)
#         f.set_size_inches((12,12))
#         plt.subplots_adjust(wspace=0.15, hspace=0.15)   # subplot간의 간격 wspace(좌우), hspace(상하간격)

#         # figure 전체 제목
#         f.subtitle('Training step result', fontsize = 15)
#         # [0,0] 위치 MAML의 train, test loss 비교
#         axes[0,0].plot(MAML_loss_tr,label='train_loss')
#         axes[0,0].plot(MAML_loss_te,label='test_loss')
#         plt.legend()
#         plt.title('MAML')
#         plt.xlabel('epochs')
#         plt.ylabel('loss')
#         # [0,1] 위치 NonMAML의 train, test loss 비교
#         axes[0,1].plot(pre_loss_tr,label='train_loss')
#         axes[0,1].plot(pre_loss_te,label='test_loss')
#         plt.legend()
#         plt.title('pretrain')
#         plt.xlabel('epochs')
#         plt.ylabel('loss')        
#         # [1,0] 위치 MAML과 NonMAML의 test loss 비교
#         axes[1,0].plot(MAML_loss_te, label='MAML')
#         axes[1,0].plot(pre_loss_te, label='pretrain')        
#         plt.legend()
#         plt.title('test_loss - MAML & pretreain')
#         plt.xlabel('epochs')
#         plt.ylabel('loss')            
#         # [1,1] 위치 MAML과 NonMAML의 r2 score 비교
#         axes[1,1].plot(MAML_r2, label='MAML')
#         axes[1,1].plot(pre_r2, label='pretrain')        
#         plt.legend()
#         plt.title('R2 score - MAML & pretreain')
#         plt.xlabel('epochs')
#         plt.ylabel('R2')     
        
#         plt.show()

# plot_training_step_result(maml.meta_losses_tr, maml.meta_losses_te, maml.r2_score, 
#                           pretrained_model.losses_tr, pretrained_model.losses_te, pretrained_model.r2_score)

## Testing step and Plotting

Now that the model is trained, let's look at how it performs, and compare it

In [ ]:
class test_step():
    # def __init__(self, trained_model, train_dataloader, test_dataloader, torch_load_filename, epochs=10, repeat_n=5, optim=torch.optim.SGD, dot=True):
    def __init__(self, pt_file_path, torch_load_filename, train_dataloader, test_dataloader, epochs=10, repeat_n=5, optim=torch.optim.SGD, dot=True):            
        # self.trained_model = trained_model
        self.tr_DL = train_dataloader
        self.te_DL =test_dataloader
        self.criterion = nn.MSELoss()
        self.optim = optim
        self.epochs = epochs
        # self.batch_num = len(self.tr_DL[0])     # 데이터로더에 있는 배치 개수
        self.batch_num = repeat_n     # 평균 구하기 위해 반복하길 원하는 횟수 n
        self.wk_num = len(self.tr_DL)            # 데이터셋에 있는 workload 개수
        self.dot =dot
        self.lamb = 0.6

        # result
        self.avg_losses_tr = [ [0]*self.epochs ]*self.wk_num
        self.avg_losses_te = [ [0]*self.epochs ]*self.wk_num
        self.avg_r2_score = [ [0]*self.epochs ]*self.wk_num
        self.MODEL_PATH = pt_file_path
        self.MODEL_NAME = torch_load_filename

    def test_on_workload(self):
        model = ReshapeNet(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, group_dim=group_dim, wk_vec_dim=wk_vec_dim).cuda()
        # MODEL_PATH = 'model_save'
        # MODEL_NAME = self.torch_load_filename
        model.load_state_dict(torch.load(os.path.join(self.MODEL_PATH, self.MODEL_NAME)))
        # model.load_state_dict(self.trained_model.state_dict())
        optimizer = self.optim(model.parameters(), 0.01)    # need to tuning

        losses_tr = []
        losses_te = []
        r2_score = []

        X, y = self.sample_tr[0], self.sample_tr[1]

        for e in range(self.epochs):
            optimizer.zero_grad()

            pred = model(X)
            if self.dot:
                # d = torch.bmm(self.model.p_res_x, self.model.p_res_x.transpose(1, 2))
                d = torch.bmm(model.res_x, model.res_x.transpose(1, 2))
                dot_loss = F.mse_loss(d, torch.eye(d.size(1)).repeat(X.shape[0], 1, 1).cuda())
                # loss = (1-self.lamb)*self.criterion(model(X), y) + self.lamb*dot_loss
                loss = (1-self.lamb)*self.criterion(pred, y) + self.lamb*dot_loss

            else:
                loss = self.criterion(model(X), y)
            losses_tr.append(loss.item())
            
            loss.backward()
            optimizer.step()

            loss_te, te_ouputs, r2_res = self.validate(model)
            # losses_te += loss_te
            # r2_score += r2_res
            losses_te.append(loss_te) 
            r2_score.append(r2_res)
            # print(loss_te)
            # print(losses_te)           

            
        return losses_tr, losses_te, r2_score         

    def average_result(self):
        time_ = datetime.datetime.today().strftime('%y%m%d/%H:%M:%S')
        print(f"[{time_}] train start")
        for wk in range(self.wk_num):
            # sampler_tr = Sampler(dataloaders=self.tr_DL[wk])   
            sampler_tr = iter(self.tr_DL[wk])            
         
            for n_batch in range(self.batch_num):   # 데이터로더의 배치 수 만큼           
                  # len(self.tr_DL): workload개수
                sample_tr = next(sampler_tr)        # wk번째 데이터 로드의 다음 배치데이터           
                self.sample_tr = sample_tr    
                self.te_DL_wk = self.te_DL[wk]

                losses_tr, losses_te, r2_core = self.test_on_workload()
                self.avg_losses_tr[wk] = [l + l_new for l, l_new in zip(self.avg_losses_tr[wk], losses_tr)]
                self.avg_losses_te[wk] = [l + l_new for l, l_new in zip(self.avg_losses_te[wk], losses_te)]
                self.avg_r2_score[wk] = [r + r_new for r, r_new in zip(self.avg_r2_score[wk], r2_core)]
                
                #################################################################################
                self.temp_avg_losses_tr[wk] = [l / n_batch for l in self.avg_losses_tr[wk]]
                self.temp_avg_losses_te[wk] = [l / n_batch for l in self.avg_losses_te[wk]]
                self.temp_avg_r2_score[wk] = [l / n_batch for l in self.avg_r2_score[wk]]
                #################################################################################
                # print('self.avg_losses_tr : -------------------------')
                # print(self.avg_r2_score)
                # print('self.avg_losses_tr[wk] : -------------------------')
                # print(self.avg_r2_score[wk])
            time_ = datetime.datetime.today().strftime('%y%m%d/%H:%M:%S')
            print(f'[{time_}] workload {wk+1}번째 끝')
            self.avg_losses_tr[wk] = [l / self.batch_num for l in self.avg_losses_tr[wk]]
            self.avg_losses_te[wk] = [l / self.batch_num for l in self.avg_losses_te[wk]]
            self.avg_r2_score[wk] = [l / self.batch_num for l in self.avg_r2_score[wk]]

            # print('평균 낸 후 self.avg_losses_tr[wk] : -------------------------')
            # # print(self.avg_r2_score[wk])
                       

        return self.avg_losses_tr, self.avg_losses_te, self.avg_r2_score

        # for n_batch in range(self.batch_num):  # 데이터로더의 배치 수 만큼
        #     sample_tr = sampler_tr.get_sample()
        #     for wk in range(self.wk_num):      # len(self.tr_DL): workload개수
        #         self.sample_tr = sample_tr[wk]    
        #         self.te_DL_wk = self.te_DL[wk]            
        #         losses_tr, losses_te, r2_core = self.test_on_workload()
        #         avg_losses_tr = [l + l_new for l, l_new in zip(avg_losses_tr, losses_tr)]
        #         avg_losses_te = [l + l_new for l, l_new in zip(avg_losses_te, losses_te)]
        #         avg_r2_score = [r + r_new for r, r_new in zip(avg_r2_score, r2_core)]
        # avg_losses_tr = [l / self.batch_num for l in avg_losses_tr]
        # avg_losses_te = [l / self.batch_num for l in avg_losses_te]    
        # avg_r2_score = [l / self.batch_num for l in avg_r2_score]        

        # return avg_losses_tr, avg_losses_te, avg_r2_score

    def validate(self, model):  ###
        model.eval()

        total_loss = 0
        total_dot_loss = 0        
        outputs = torch.Tensor().cuda()
        r2_res = 0
        with torch.no_grad():
            for data, target in self.te_DL_wk:
                output = model(data)
                if self.dot:
                    d = torch.bmm(model.res_x, model.res_x.transpose(1, 2))
                    dot_loss = F.mse_loss(d, torch.eye(d.size(1)).repeat(data.shape[0], 1, 1).cuda())
                    loss = (1-self.lamb)*F.mse_loss(output, target) + self.lamb*dot_loss
                else:
                    dot_loss = 0
                    loss = F.mse_loss(output, target)
                true = target.cpu().detach().numpy().squeeze()
                pred = output.cpu().detach().numpy().squeeze()
                r2_res += r2_score(true, pred)
                total_loss += loss.item()
                total_dot_loss += dot_loss.item()
                outputs = torch.cat((outputs, output))
        total_loss /= len(self.te_DL_wk) #* len(self.te_DL)
        total_dot_loss /= len(self.te_DL_wk) #* len(self.te_DL)
        r2_res /= len(self.te_DL_wk) #* len(self.te_DL)

        return total_loss, outputs, r2_res

Make tesing step dataloader

In [ ]:
'''preparing for tesing step dataset'''

wk_type = {0:'WR 9:1 (1)',  1:'WR 9:1 (4)',  2:'WR 9:1 (16)',  3:'WR 9:1 (64)',
           4:'WR 1:1 (1)',  5:'WR 1:1 (4)',  6:'WR 1:1 (16)',  7:'WR 1:1 (64)',
           8:'WR 1:9 (1)',  9:'WR 1:9 (4)',  10:'WR 1:9 (16)', 11:'WR 1:9 (64)',
           12:'UPdate (1)', 13:'UPdate (4)', 14:'UPdate (16)', 15:'UPdate (64)'}

K_te = 10   # batch size of testing step
wk_using_testing_step = [2,6,10,14]

def make_wk_type_using(wk_using_testing_step):    
    wk_type_using = []
    for i in range(len(wk_using_testing_step)):    
        wk_type_using.append(wk_type[wk_using_testing_step[i]])

wk_type_using = make_wk_type_using(wk_using_testing_step)

# # for dataloader new task test(MAML, pretrain)
New_task_DL_tr, New_task_DL_te, New_task_X_te, New_task_y_te = MAML_dataset(entire_X_tr, entire_y_tr, entire_X_te, entire_y_te, scaler_X, scaler_y, wk_using_testing_step, batch_size=K_te)

r: 빨강
g: 초록
b: 파랑
black : 검정
c: 시안
m: 마젠타

wk_vec = [[9,1,0,1], [9,1,0,4], [9,1,0,16], [9,1,0,64],
            [1,1,0,1], [1,1,0,4], [1,1,0,16], [1,1,0,64],
            [1,9,0,1], [1,9,0,4], [1,9,0,16], [1,9,0,64],
            [0,0,1,1], [0,0,1,4], [0,0,1,16], [0,0,1,64]]

wk_using_training_step = [0,1,3,
                          4,5,7,
                          8,9,11,
                          12,13,15]

wk_using_test_step = [2,6,10,14]

In [ ]:
# MAML_avg_losses_tr, MAML_avg_losses_te, MAML_avg_r2_score = test_step('model_save',name_MAML_one_batch_pt, New_task_DL_tr, New_task_DL_te, epochs=10, repeat_n=100).average_result()
MAML_avg_losses_tr, MAML_avg_losses_te, MAML_avg_r2_score = test_step('model_save', '20220425/MAML_pretrain-20220424-02.p', New_task_DL_tr, New_task_DL_te, epochs=10, repeat_n=100).average_result()


In [ ]:
# pre_avg_losses_tr, pre_avg_losses_te, pre_avg_r2_score = test_step('model_save',name_nonMAML_pretrain_pt, New_task_DL_tr, New_task_DL_te, epochs=10, repeat_n=100).average_result()
pre_avg_losses_tr, pre_avg_losses_te, pre_avg_r2_score = test_step('model_save','20220425/pretrain-20220424-03.pt', New_task_DL_tr, New_task_DL_te, epochs=10, repeat_n=100).average_result()

Testing step result plotting

maml.meta_losses_tr
maml.meta_losses_te
maml.r2_score
        
pretrained_model.losses_tr
pretrained_model.losses_te
pretrained_model.r2_score

MAML_avg_losses_tr
MAML_avg_losses_te
MAML_avg_r2_score

pre_avg_losses_tr
pre_avg_losses_te
pre_avg_r2_score

In [ ]:
plt.plot(MAML_avg_losses_te[0], label='WR9:1 (16)')
plt.plot(MAML_avg_losses_te[1], label='WR1:1 (16)')
plt.plot(MAML_avg_losses_te[2], label='WR1:9 (16)')
plt.plot(MAML_avg_losses_te[3], label='Update(16)')
plt.xlabel('epochs')
plt.ylabel('loss_te')
plt.title('Testing step_MAML (loss_te)')
plt.legend()

In [ ]:
plt.plot(pre_avg_losses_te[0], label='WR9:1 (16)')
plt.plot(pre_avg_losses_te[1], label='WR1:1 (16)')
plt.plot(pre_avg_losses_te[2], label='WR1:9 (16)')
plt.plot(pre_avg_losses_te[3], label='Update(16)')
plt.xlabel('epochs')
plt.ylabel('loss_te')
plt.title('Testing step_pre (loss_te)')
plt.legend()

In [ ]:
plt.plot(MAML_avg_r2_score[0], label='WR9:1 (16)')
plt.plot(MAML_avg_r2_score[1], label='WR1:1 (16)')
plt.plot(MAML_avg_r2_score[2], label='WR1:9 (16)')
plt.plot(MAML_avg_r2_score[3], label='Update(16)')
plt.xlabel('epochs')
plt.ylabel('r2-score')
plt.title('Testing step_MAML (r2) ')
plt.legend()

In [ ]:
plt.plot(pre_avg_r2_score[0], label='WR9:1 (16)')
plt.plot(pre_avg_r2_score[1], label='WR1:1 (16)')
plt.plot(pre_avg_r2_score[2], label='WR1:9 (16)')
plt.plot(pre_avg_r2_score[3], label='Update(16)')
plt.xlabel('epochs')
plt.ylabel('r2-score')
plt.title('Testing step_pre (r2)')
plt.legend()

In [ ]:
plt.plot(pre_avg_losses_te[0], label='P_WR9:1 (16)')
plt.plot(pre_avg_losses_te[1], label='P_WR1:1 (16)')
plt.plot(pre_avg_losses_te[2], label='P_WR1:9 (16)')
plt.plot(pre_avg_losses_te[3], label='P_Update(16)')

plt.plot(MAML_avg_losses_te[0], label='M_WR9:1 (16)')
plt.plot(MAML_avg_losses_te[1], label='M_WR1:1 (16)')
plt.plot(MAML_avg_losses_te[2], label='M_WR1:9 (16)')
plt.plot(MAML_avg_losses_te[3], label='M_Update(16)')
plt.xlabel('epochs')
plt.ylabel('loss_te')
plt.title('Testing step_MAML & pre (loss_te) ')
plt.legend()

In [ ]:
plt.plot(pre_avg_r2_score[0], label='P_WR9:1 (16)')
plt.plot(pre_avg_r2_score[1], label='P_WR1:1 (16)')
plt.plot(pre_avg_r2_score[2], label='P_WR1:9 (16)')
plt.plot(pre_avg_r2_score[3], label='P_Update(16)')

plt.plot(MAML_avg_r2_score[0], label='M_WR9:1 (16)')
plt.plot(MAML_avg_r2_score[1], label='M_WR1:1 (16)')
plt.plot(MAML_avg_r2_score[2], label='M_WR1:9 (16)')
plt.plot(MAML_avg_r2_score[3], label='M_Update(16)')
plt.xlabel('epochs')
plt.ylabel('r2-score')
plt.title('Testing step_MAML & pre (r2) ')
plt.legend()

In [ ]:
# def plot_training_step_result(wk_type_using, MAML_avg_losses_tr, MAML_avg_losses_te, MAML_avg_r2_score, pre_avg_losses_tr, pre_avg_losses_te, pre_avg_r2_score, wk_type_using):
#         wk_num = len(MAML_avg_losses_tr)
#         # 그래프 배열 지정
#         f, axes = plt.subplots(3,2)
#         f.set_size_inches((12,12))
#         plt.subplots_adjust(wspace=0.15, hspace=0.15)   # subplot간의 간격 wspace(좌우), hspace(상하간격)

#         # figure 전체 제목
#         f.subtitle('Testing step result', fontsize = 15)
#         # [0,0] 위치 MAML의 testing step에서의 test loss 비교
#         for i in range(wk_num):
#                 axes[0,0].plot(MAML_avg_losses_te[i],label=f'{wk_type_using[i]}')
#         axes[0,0].set_tile('MAML loss_te')
#         plt.legend()
#         # plt.title('MAML')
#         plt.xlabel('epochs')
#         plt.ylabel('loss')
#         # [0,1] 위치 NonMAML의 testing step에서의 test loss 비교
#         for i in range(wk_num):
#                 axes[0,0].plot(pre_avg_losses_te[i],label=f'{wk_type_using[i]}')
#         axes[0,0].set_tile('pre loss_te')
#         plt.legend()
#         # plt.title('MAML')
#         plt.xlabel('epochs')
#         plt.ylabel('loss')

# #       plt.xlabel('epochs')가 아니라 axes[?,?].set_xlabel()을 해야하나?






        
#         plt.show()

# plot_training_step_result(maml.meta_losses_tr, maml.meta_losses_te, maml.r2_score, 
#                           pretrained_model.losses_tr, pretrained_model.losses_te, pretrained_model.r2_score)